HOMEWORK 4

VARSHA KARANAM

VEDA CHARITHA BELLAM

PRADYUMNA KAPUTHIMAREDDY



1)Create your own dataset for text classification. It should contain at least 2000 words in total and at least three categories with at least 100 examples per category (an example can be a poem or a paragraph from a book). You can create it by scraping the web or using some of the documents you have on your computer (do not use anything confidential) or ChatGPT.

In [7]:
import pandas as pd


categories = ['Nature', 'Technology', 'Literature']
nature_examples = [
    "The gentle rustle of leaves in the breeze whispered tales of ancient forests and forgotten legends to those who listened.",
    "As dawn broke, the first rays of sunlight kissed the dew-covered grass, painting the meadow in a golden hue.",
    "Butterflies flitted gracefully from flower to flower, their delicate wings shimmering in the morning light.",
    "The tranquil melody of a babbling brook echoed through the valley, soothing the weary souls of travelers.",
    "Beneath the canopy of stars, the nocturnal creatures of the forest danced to the rhythm of the night, their silhouettes weaving intricate patterns in the darkness.",
    "A gentle rain shower bathed the earth in a cleansing embrace, nourishing the soil and awakening dormant seeds.",
    "The majestic oak tree stood tall and proud, its branches reaching towards the heavens in silent reverence.",
    "Fields of vibrant wildflowers stretched as far as the eye could see, painting the landscape in a kaleidoscope of colors.",
    "A family of deer grazed peacefully in the meadow, their graceful movements adding to the serene beauty of the landscape.",
    "The scent of pine trees hung heavy in the air, mingling with the sweet aroma of wildflowers, as the forest whispered secrets of times long past."
]

technology_examples = [
    "In the bustling city streets, neon lights illuminated the faces of passersby, casting an otherworldly glow upon the urban landscape.",
    "The whirring of gears and the rhythmic clanking of machinery filled the air of the industrial district, a symphony of progress and innovation.",
    "Holographic displays flickered to life, projecting holograms of futuristic cities and advanced technology into the room.",
    "Programmers meticulously crafted lines of code, each keystroke bringing their digital creations one step closer to reality.",
    "Autonomous vehicles zipped through the cityscape, navigating the bustling streets with precision and efficiency, a testament to human ingenuity.",
    "The soft glow of a computer screen illuminated the dimly lit room, as a programmer worked tirelessly into the night, bringing their vision to life.",
    "Nanobots swarmed through the bloodstream, repairing damaged tissue and fighting off infections with pinpoint accuracy.",
    "The hum of drones filled the air as they soared overhead, capturing breathtaking aerial footage of the cityscape, their cameras recording every detail with stunning clarity.",
    "Artificial intelligence algorithms analyzed vast datasets, uncovering hidden patterns and insights that eluded human comprehension.",
    "Quantum computers revolutionized the field of computing, solving complex problems with unprecedented speed and efficiency, unlocking new possibilities in fields ranging from cryptography to drug discovery."
]

literature_examples = [
    "In the heart of the enchanted forest, a young heroine embarked on a quest to unravel the mysteries of an ancient prophecy and save her kingdom from darkness.",
    "\"It was the best of times, it was the worst of times,\" began the timeless tale of love, sacrifice, and redemption set against the backdrop of revolution.",
    "The haunting melody of a violin echoed through the halls of the abandoned mansion, a ghostly reminder of the tragic love story that unfolded within its walls.",
    "Amidst the ruins of a once-great empire, a band of rebels rose up against tyranny, their rallying cry echoing across the desolate landscape.",
    "From the ashes of war, a new era of peace and prosperity dawned, as heroes and heroines alike forged a brighter future for generations to come.",
    "The pages of an ancient tome crackled with magic as a young wizard delved into its secrets, seeking knowledge that would change the course of history.",
    "A mysterious stranger wandered into town, his cloak billowing behind him as he whispered tales of adventure and intrigue to anyone who would listen.",
    "In the depths of the ocean, a mermaid sang a haunting melody, her voice luring sailors to their doom as they searched for the lost city of Atlantis.",
    "The castle loomed ominously on the hill, its darkened windows hiding secrets of betrayal and revenge that spanned centuries.",
    "As the sun set on the horizon, a lone figure stood on the cliff's edge, gazing out at the endless expanse of the sea, lost in thoughts of love and loss."
]



data = pd.DataFrame(columns=['Text', 'Category'])


for category, examples in zip(categories, [nature_examples, technology_examples, literature_examples]):
    for example in examples:
        data = pd.concat([data, pd.DataFrame({'Text': [example], 'Category': [category]})], ignore_index=True)


data = data.sample(frac=1).reset_index(drop=True)


data.to_csv('text_dataset.csv', index=False)

print("Dataset created successfully!")



Dataset created successfully!


This code creates a synthetic dataset of text examples categorized into 'Nature', 'Technology', and 'Literature'. It defines example sentences for each category, creates a DataFrame to store the data, populates the DataFrame with examples, shuffles the dataset, saves it to a CSV file.

In [8]:
from sklearn.model_selection import train_test_split


min_train_examples = 240
min_test_examples = 60

train_data, test_data = train_test_split(data, test_size=0.2, stratify=data['Category'])


train_data = train_data.groupby('Category').apply(lambda x: x.sample(min(min_train_examples, len(x)))).reset_index(drop=True)


test_data = test_data.groupby('Category').apply(lambda x: x.sample(min(min_test_examples, len(x)))).reset_index(drop=True)


train_data = train_data.sample(frac=1).reset_index(drop=True)
test_data = test_data.sample(frac=1).reset_index(drop=True)


train_data.to_csv('train_dataset.csv', index=False)
test_data.to_csv('test_dataset.csv', index=False)

print("Training and test datasets created successfully!")


Training and test datasets created successfully!


In [15]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW
from torch.nn.utils.rnn import pad_sequence


model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained(model_name)


class CustomDataset(Dataset):
    def __init__(self, tokenizer, data_file, block_size):
        self.examples = []
        with open(data_file, "r") as f:
            lines = f.readlines()
            for line in lines:
                self.examples.append(tokenizer.encode(line.strip()))

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        return torch.tensor(self.examples[idx])


num_train_epochs = 11
learning_rate = 5e-5
batch_size = 4


train_dataset = CustomDataset(tokenizer, "train_dataset.csv", block_size=128)


train_dataset_tensors = [torch.tensor(example) for example in train_dataset.examples]

padded_train_dataset = pad_sequence(train_dataset_tensors, batch_first=True, padding_value=tokenizer.pad_token_id)


train_loader = DataLoader(padded_train_dataset, batch_size=batch_size, shuffle=True)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = AdamW(model.parameters(), lr=learning_rate)

for epoch in range(num_train_epochs):
    model.train()
    total_loss = 0.0
    for batch in train_loader:
        inputs = batch.to(device)
        outputs = model(inputs, labels=inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()
    print(f"Epoch {epoch + 1} Loss: {total_loss}")


model.save_pretrained("gpt2-finetuned")

print("Fine-tuning completed!")




/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1 Loss: 30.896241903305054
Epoch 2 Loss: 15.595089435577393
Epoch 3 Loss: 12.065539956092834
Epoch 4 Loss: 10.006148219108582
Epoch 5 Loss: 7.617909789085388
Epoch 6 Loss: 6.397918164730072
Epoch 7 Loss: 5.187341272830963
Epoch 8 Loss: 3.8969529569149017
Epoch 9 Loss: 2.973611533641815
Epoch 10 Loss: 2.4432801604270935
Epoch 11 Loss: 2.0208991318941116
Fine-tuning completed!


This code fine-tunes a pre-trained GPT-2 language model on a custom dataset by first loading and preprocessing the training data, then creating a DataLoader for efficient batch processing. The model undergoes training for a specified number of epochs, where each iteration involves computing the loss between model predictions and actual data, backpropagating gradients, and updating model parameters using the AdamW optimizer. After training, the fine-tuned model is saved to disk, enabling its use for various natural language processing tasks such as text generation or completion.

We used perplexity as accuracy metric. Test perplexity is a measure of how well a language model predicts sequences of tokens (like words or characters) on unseen data. Lower perplexity indicates better performance, meaning the model's predictions align more closely with the actual data. It's calculated as the exponentiation of the average cross-entropy loss, with higher values suggesting greater uncertainty or less accurate predictions.

In [14]:

test_dataset = CustomDataset(tokenizer, "test_dataset.csv", block_size=128)


test_dataset_tensors = [torch.tensor(example) for example in test_dataset.examples]


padded_test_dataset = pad_sequence(test_dataset_tensors, batch_first=True, padding_value=tokenizer.pad_token_id)


test_loader = DataLoader(padded_test_dataset, batch_size=batch_size)


model.eval()
total_loss = 0.0
num_batches = 0

with torch.no_grad():
    for batch in test_loader:
        inputs = batch.to(device)
        outputs = model(inputs, labels=inputs)
        loss = outputs.loss
        total_loss += loss.item()
        num_batches += 1


test_perplexity = torch.exp(torch.tensor(total_loss / num_batches))
print("Test Perplexity:", test_perplexity.item())


Test Perplexity: 13.89050579071045


The test perplexity is 13.8905

To increase the accuracy we can perform the following steps:




*   Grid search or random search over hyperparameters might help find better combinations.
*   Instead of using a fixed learning rate, try scheduling the learning rate to decrease over time. Techniques like learning rate warm-up or using a learning rate scheduler can sometimes improve convergence.

*   Explore different loss functions or modify the existing loss function to better suit the task or model architecture.






In [ ]:
!pip install transformers

In [ ]:
!pip install transformers accelerate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 3.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

REFERENCES:

https://huggingface.co/openai-community/gpt2

https://towardsdatascience.com/natural-language-generation-part-2-gpt-2-and-huggingface-f3acb35bc86a

CHATGPT